In [1]:
from config import *
df = get_prepared_df()

In [74]:
show_structure(indices=90844, web=True)

Structure web url: https://rnaloops.cs.put.poznan.pl/search/details/90844


/home/lukas/Projects/RNALoops/rnaloops/data_explorer/explore_fcts.py:161: ResourceWarning: unclosed <socket.socket fd=52, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 35824), raddr=('127.0.0.1', 58021)>
  init_driver(url=url, headless=False, detached=True)


,loop_type,home_structure,db_notation,whole_sequence,helix_1_bps,helix_2_bps,helix_3_bps,helix_4_bps,helix_5_bps,helix_6_bps,helix_7_bps,helix_8_bps,helix_9_bps,helix_10_bps,helix_11_bps,helix_12_bps,helix_13_bps,helix_14_bps,strand_1_nts,strand_2_nts,strand_3_nts,strand_4_nts,strand_5_nts,strand_6_nts,strand_7_nts,strand_8_nts,strand_9_nts,strand_10_nts,strand_11_nts,strand_12_nts,strand_13_nts,strand_14_nts,end_1,end_2,end_3,end_4,end_5,end_6,end_7,end_8,end_9,end_10,end_11,end_12,end_13,end_14,euler_x_1,euler_x_2,euler_x_3,euler_x_4,euler_x_5,euler_x_6,euler_x_7,euler_x_8,euler_x_9,euler_x_10,euler_x_11,euler_x_12,euler_x_13,euler_x_14,start_1,start_2,start_3,start_4,start_5,start_6,start_7,start_8,start_9,start_10,start_11,start_12,start_13,start_14,euler_y_1,euler_y_2,euler_y_3,euler_y_4,euler_y_5,euler_y_6,euler_y_7,euler_y_8,euler_y_9,euler_y_10,euler_y_11,euler_y_12,euler_y_13,euler_y_14,euler_z_1,euler_z_2,euler_z_3,euler_z_4,euler_z_5,euler_z_6,euler_z_7,euler_z_8,euler_z_9,euler_z_10,euler_z_11,euler_z_12,euler_z_13,euler_z_14,planar_1,planar_2,planar_3,planar_4,planar_5,planar_6,planar_7,planar_8,planar_9,planar_10,planar_11,planar_12,planar_13,planar_14,sequence_1,sequence_2,sequence_3,sequence_4,sequence_5,sequence_6,sequence_7,sequence_8,sequence_9,sequence_10,sequence_11,sequence_12,sequence_13,sequence_14,sequence_15,sequence_16,sequence_17,sequence_18,sequence_19,sequence_20,sequence_21,sequence_22,sequence_23,sequence_24,sequence_25,sequence_26,sequence_27,sequence_28
90844,03-way,3jbu,(((((..(( — )).((((( — ))))).....))))),AUCACAUCC — GGUAUGGC — GCCAUUUAAAGUGGU,5,2,5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,1,5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,b-2519,b-2546,b-2566,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,64.193001,178.932007,116.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b-2518,b-2546,b-2562,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,123.181,150.136002,86.681999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.975998,133.522995,144.501007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.111,153.386993,111.667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AUCAC-GUGGU,AU,CC-GG,U,AUGGC-GCCAU,UUAAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
structures = load_full_structures(1, 1)

In [22]:
90716

74

In [71]:
def get_helices(ml):
    
    considered_bonds = 'WATSON-CRICK', 'G-C PAIR', 'TYPE_28_PAIR', 'C-G PAIR'
    
    n = int(s.multiloops.loc[ml].loop_type.split('-')[0])
    keys = [f'helix{i}' for i in range(1, 1 + n)]
    ml_helices = []
    
    indices = [n] + list(range(1, n))
    
    for counter, idx in enumerate(indices):

        end = s.multiloops.loc[ml, f'end_{idx}']
        expected_length = s.multiloops.loc[ml, f'helix_{counter+1}_bps']
        skipped = 0
        
        while end == '-':
            
            skipped += 1
            
            if skipped > n:
                print(f'Warning: Cant get helices of id {ml}, has no strands!')
                return pd.concat([s.bonds_df.iloc[0:0]], keys=keys)
            
            idx = idx + 1 if idx < n else 1
            end = s.multiloops.loc[ml, f'start_{idx}']
        
        next_start_prev = base_order.at[end, 'prev']
        
        for _ in range(skipped):
            next_start_prev_idx = s.bonds_df.loc[next_start_prev].index
            next_start_prev = next_start_prev_idx.get_level_values(0).values[0]

        next_base = base_order.at[end, 'next']

        helix_bases = []

        while any(bond in s.bonds_df.loc[next_base].kind.values 
                  for bond in considered_bonds):
            helix_bases.append(next_base)
            next_base = base_order.at[next_base, 'next']
            
        post = 'base pairs than in RNALoops!'
        bps_diff = len(helix_bases) - expected_length
        if bps_diff != 0:
            print(f'Warning: #bps differs for id {ml} {keys[counter]} by',
                  bps_diff)
        else:
            print(ml)

        ml_helices.append(s.bonds_df.loc[helix_bases])

    return pd.concat(ml_helices, keys=keys)

In [72]:
base_order = s.get_base_order()
helices = []

for j, ml in enumerate(s.multiloops.index):
    helices.append(get_helices(ml))
structure_helices = pd.concat(helices, keys=s.multiloops.index)

90774
90774
90774
90774
90715
90803
90797
90844
90844
90844
90777
90777
90777
90781
90781
90781
90781
90736
90790
90790
90716
90881
90881
90710
90709
90709
90837
90837
90837
90742
90742
90742
90842
90713
90713
90830
90830
90753
90753
90820
90820
90759
90759
90759
90718
90718
90768
90768
90768
90762
90762
90762
90795
90795
90795
90800
90800
90800
90800
90760
90760
90760
90816
90816
90816
90766
90758
90758
90810
90810
90810
90846
90846
90782
90787
90787
90808
90779
90779
90779
90734
90732
90732
90732
90755
90755
90880
90880
90880
90724
90724
90724
90712
90761
90827
90827
90827
90754
90754
90754
90730
90730
90711
90721
90721
90839
90839
90818
90818
90792
90792
90707
90707
90707
90751
90834
90834
90746
90746
90746
90746
90848
90848
90848
90848
90726
90728
90728
90794
90794
90771
90771
90784
90784
90780
90780
90744
90744
90744
90813
90813
90813
90813
90813
90836
90836
90770
90770


In [38]:
structure_helices.head(200)

type          kind
             pos1   pos2   atom1 atom2 ptnr1 ptnr2                      
90774 helix1 b-604  b-624  O6    N4    G     C      hydrog  WATSON-CRICK
                           N2    O2    G     C      hydrog  WATSON-CRICK
                           N1    N3    G     C      hydrog  WATSON-CRICK
             b-605  b-623  O6    N4    G     C      hydrog  WATSON-CRICK
                           N1    N3    G     C      hydrog  WATSON-CRICK
                           N2    O2    G     C      hydrog  WATSON-CRICK
             b-606  b-622  N3    O6    U     G      hydrog  TYPE_28_PAIR
                           O2    N1    U     G      hydrog  TYPE_28_PAIR
      helix2 b-628  b-635  N2    O2    G     C      hydrog  WATSON-CRICK
                           O6    N4    G     C      hydrog  WATSON-CRICK
                           N1    N3    G     C      hydrog  WATSON-CRICK
             b-629  b-634  O6    N4    G     C      hydrog  WATSON-CRICK
                           N2    O2    G     C      hydrog  WATSON-CRICK
                           N1    N3    G     C      hydrog  WATSON-CRICK
      helix3 b-638  b-649  N1    O6    G     G      hydrog   G-G MISPAIR
                    b-650  N1    N3    G     C      hydrog  WATSON-CRICK
                           N2    O2    G     C      hydrog  WATSON-CRICK
                           O6    N4    G     C      hydrog  WATSON-CRICK
             b-639  b-649  N3    O6    U     G      hydrog  TYPE_28_PAIR
                           O2    N1    U     G      hydrog  TYPE_28_PAIR
             b-640  b-648  N3    N1    C     G      hydrog  WATSON-CRICK
                           N4    O6    C     G      hydrog  WATSON-CRICK
                           O2    N2    C     G      hydrog  WATSON-CRICK
             b-641  b-647  N3    O6    U     G      hydrog  TYPE_28_PAIR
                           O2    N1    U     G      hydrog  TYPE_28_PAIR
      helix4 b-656  b-600  N1    O6    G     G      hydrog   G-G MISPAIR
                    b-601  N2    O2    G     C      hydrog  WATSON-CRICK
                           O6    N4    G     C      hydrog  WATSON-CRICK
                           N1    N3    G     C      hydrog  WATSON-CRICK
90714 helix2 A-577  A-764  N1    N3    G     C      hydrog  WATSON-CRICK
                           N2    O2    G     C      hydrog  WATSON-CRICK
                           O6    N4    G     C      hydrog  WATSON-CRICK
                    A-729  N2    N3    G     A      hydrog   G-A MISPAIR
      helix3 A-769  A-810  N1    N3    G     C      hydrog  WATSON-CRICK
                           N2    O2    G     C      hydrog  WATSON-CRICK
                           O6    N4    G     C      hydrog  WATSON-CRICK
                    A-900  N2    N1    G     A      hydrog   G-A MISPAIR
             A-770  A-809  N3    N1    C     G      hydrog  WATSON-CRICK
                           N4    O6    C     G      hydrog  WATSON-CRICK
                           O2    N2    C     G      hydrog  WATSON-CRICK
                    A-899  O2    N4    C     C      hydrog   C-C MISPAIR
             A-771  A-808  N1    O2    G     C      hydrog      G-C PAIR
      helix4 A-821  A-879  O6    N4    G     C      hydrog  WATSON-CRICK
                           N2    O2    G     C      hydrog  WATSON-CRICK
                           N1    N3    G     C      hydrog  WATSON-CRICK
             A-822  A-878  O4    N6    U     A      hydrog  WATSON-CRICK
                           N3    N1    U     A      hydrog  WATSON-CRICK
      helix5 A-865  A-570  N6    O6    A     G      hydrog   G-A MISPAIR
                    A-571  N1    N3    A     U      hydrog  WATSON-CRICK
                           N6    O4    A     U      hydrog  WATSON-CRICK
             A-866  A-570  O2    N2    C     G      hydrog  WATSON-CRICK
                           N4    O6    C     G      hydrog  WATSON-CRICK
                           N3    N1    C     G      hydrog  WATSON-CRICK
             A-867  A-862  N1    N3    